In [2]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
burned = pd.read_csv("src/burnedsampleData.csv")
unburned = pd.read_csv("src/unburnedsampleData.csv")

In [4]:
burned.head()

,FIRE_NAME,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,...,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_sagebrush_height,rangeland_shrub,rangeland_shrub_height,x_coord,y_coord
0,ATLAS,7901.0,8141,8855,9145,13120,12799,10556,1,9.171320,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185765.625002,36129.374898
1,ATLAS,7936.0,8119,8861,9040,13384,11922,9801,1,11.142816,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185776.875002,35589.374898
2,ATLAS,8341.0,8542,9265,9723,17122,13685,11178,1,14.236607,...,6.0,0.0,53.0,7.0,0.0,0.0,27.0,48.0,-185776.875002,35994.374898
3,ATLAS,8833.0,9513,11150,12624,17102,16821,12897,1,-13.036240,...,7.0,0.0,44.0,7.0,0.0,0.0,40.0,60.0,-185788.125002,36354.374898
4,ATLAS,8048.0,8233,8796,8842,13675,12234,10061,1,70.785240,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185799.375002,37546.874898


In [5]:
temp = burned.loc[:, ~burned.columns.isin(['SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
utemp = unburned.loc[:, ~unburned.columns.isin(['SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
for i in temp.columns:
    burned[i]=pd.to_numeric(temp[i], errors='coerce')
    unburned[i]=pd.to_numeric(utemp[i], errors='coerce')
df = pd.concat([unburned,burned])
df

,FIRE_NAME,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,...,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_sagebrush_height,rangeland_shrub,rangeland_shrub_height,x_coord,y_coord
0,ATLAS,7273.0,7451.0,7730,7416,7395,7397,7348,1,0.107050,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183748.125002,57165.624898
1,ATLAS,9116.0,9973.0,11849,14092,19003,18687,14163,1,-24.181034,...,24.0,0.0,52.0,11.0,0.0,0.0,6.0,15.0,-183775.625002,56436.874898
2,ATLAS,7744.0,7961.0,8575,8730,13556,11451,9479,1,-6.647168,...,6.0,0.0,8.0,24.0,0.0,0.0,59.0,158.0,-183775.625002,56904.374898
3,ATLAS,7368.0,7456.0,7593,7511,8461,7934,7579,1,99.826752,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183803.125002,54484.374898
4,ATLAS,7760.0,7843.0,8141,8340,11399,9788,8657,1,27.707573,...,4.0,0.0,25.0,10.0,0.0,0.0,41.0,141.0,-183803.125002,55804.374898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149223,WALKER,7457.0,7514.0,7745,7602,9824,8140,7737,2,107.069328,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-59833.125001,225343.124897
149224,WALKER,7497.0,7555.0,7873,7689,10645,8841,8075,2,115.286041,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-59979.375001,225320.624897
149225,WALKER,7514.0,7567.0,7967,7739,11917,9070,8154,1,68.903618,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-60181.875001,225331.874897
149226,WALKER,7457.0,7534.0,7892,7712,10404,8601,7972,1,53.100204,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-60328.125001,225455.624897


In [6]:
clean_df = df.loc[:, ~df.columns.isin(['NBR','dNBR','SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = clean_df.drop(['burnSeverity'],axis=1)
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
X_scaled

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


array([[-1.05177242, -0.98321993, -1.06533515, ...,  1.14035354,
        -0.06434863, -1.31206228],
       [ 1.20994343,  1.71555454,  2.42421759, ..., -1.47661979,
        -0.06466261, -1.31740393],
       [-0.47376474, -0.43747251, -0.3494643 , ..., -1.09592072,
        -0.06466261, -1.31397721],
       ...,
       [-0.75601902, -0.85908914, -0.86455244, ...,  1.14035354,
         1.34647285, -0.0794256 ],
       [-0.825969  , -0.89440221, -0.92809127, ...,  1.14035354,
         1.34480304, -0.07851853],
       [-0.63207257, -0.72425743, -0.67224157, ...,  1.14035354,
         1.34428925, -0.07950806]])

In [7]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LinearRegression()
pipe.fit(x_train, y_train)
print("Linear Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Linear Regression: 0.5307956750410858
SR_B1: 0.41509005645253055
SR_B2: -0.10114593085123791
SR_B3: -0.6012139956638521
SR_B6: 0.36915610067721527
NDVI: -0.4746475961812635
elevation: 0.07031161691972695
pr: -0.003921886987779202
rmax: -0.16281104098968108
rmin: -0.013977982720753837
sph: -0.2760484262278294
srad: 0.3440929993814977
th: 0.0132953099631727
tmmn: 0.17548680364632355
tmmx: 0.38417158565211534
vs: 0.06138517991643481
erc: -0.4797297847612834
eto: -1.115185629555207
bi: 0.03208446873200384
fm100: 0.07241932518934258
fm1000: -0.16341637790195973
etr: 0.778631549588585
vpd: -0.30027070333462846
landcover: 0.09175480555302098
impervious: 0.018932013925069352
percent_tree_cover: 0.27151141011028557
rangeland_annual_herbaceous: -0.21493146283753445
rangeland_bare_ground: -0.1896206494584925
rangeland_big_sagebrush: -0.5029853775497263
rangeland_herbaceous: 0.00033660704543682016
rangeland_litter: -0.12511272134983786
rangeland_sagebrush: -1.28067337864346
rangeland_sagebrush_hei

['SR_B1',
 'SR_B6',
 'elevation',
 'srad',
 'tmmn',
 'tmmx',
 'vs',
 'bi',
 'fm100',
 'etr',
 'landcover',
 'percent_tree_cover',
 'rangeland_sagebrush_height',
 'rangeland_shrub',
 'x_coord']

In [8]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LogisticRegression()
pipe.fit(x_train, y_train)
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Logistic Regression: 0.7939959909637596
SR_B1: -1.6630520740688928
SR_B2: 0.2957937792889085
SR_B3: 5.089486416736283
SR_B6: -3.9456786476864543
NDVI: 4.355687922703537
elevation: 0.553831916547396
pr: -0.11467058538224893
rmax: 0.33283709703861075
rmin: 1.0066632225495062
sph: 0.9011587397654411
srad: -0.9166047037500467
th: -0.016702504204749998
tmmn: -0.9649529576767273
tmmx: -0.11735930180295272
vs: -0.8242087005577986
erc: 0.16932110480251425
eto: 0.04865170494273265
bi: 0.6280242592278948
fm100: -0.8285605699638748
fm1000: 0.6960169240093862
etr: 0.066138217175983
vpd: -0.23013285695028318
landcover: -0.2518580759594149
impervious: -0.3696406268244394
percent_tree_cover: -2.4793123174152614
rangeland_annual_herbaceous: 0.5885852752832867
rangeland_bare_ground: 1.0072859961036429
rangeland_big_sagebrush: -0.31838742423425165
rangeland_herbaceous: 0.4655638830509066
rangeland_litter: 0.05495562728309114
rangeland_sagebrush: -0.4306998010070553
rangeland_sagebrush_height: -0.2369379

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['SR_B2',
 'SR_B3',
 'NDVI',
 'elevation',
 'rmax',
 'rmin',
 'sph',
 'erc',
 'eto',
 'bi',
 'fm1000',
 'etr',
 'rangeland_annual_herbaceous',
 'rangeland_bare_ground',
 'rangeland_herbaceous',
 'rangeland_litter',
 'x_coord',
 'y_coord']

In [9]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = RandomForestRegressor()
pipe.fit(x_train, y_train)
pipe.score(x_test, y_test)  

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.803732092251605

In [10]:
importance = pipe.feature_importances_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

SR_B1: 0.017735079650923795
SR_B2: 0.01815858451729165
SR_B3: 0.047481208026650124
SR_B6: 0.07431574176997639
NDVI: 0.3679808990971607
elevation: 0.02445306107865485
pr: 0.006390565205409445
rmax: 0.007157383075811062
rmin: 0.006012777929294912
sph: 0.01302096283981338
srad: 0.011034136827565062
th: 0.009201607212178933
tmmn: 0.0063724979109533175
tmmx: 0.013140994676009451
vs: 0.011665232495118924
erc: 0.003934114202339246
eto: 0.0049946309279195805
bi: 0.005396896737736658
fm100: 0.004405557259501325
fm1000: 0.0055525028721509495
etr: 0.004687811604331275
vpd: 0.004539835356945092
landcover: 0.023706299000647565
impervious: 0.0013905654024188715
percent_tree_cover: 0.16195013971428
rangeland_annual_herbaceous: 0.0033321110617838498
rangeland_bare_ground: 0.007361701069389731
rangeland_big_sagebrush: 0.000551367774394662
rangeland_herbaceous: 0.004119856851007256
rangeland_litter: 0.005932022430846962
rangeland_sagebrush: 0.000690707425886356
rangeland_sagebrush_height: 0.000750732647

['SR_B3',
 'SR_B6',
 'NDVI',
 'elevation',
 'landcover',
 'percent_tree_cover',
 'x_coord',
 'y_coord']

In [14]:
#keep.append('burnSeverity')
#clean_df[keep].to_csv("train.csv")